# NonlinearModel
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
source = "NonlinearExplicit";

In [ ]:
using LightGBM
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("../../TrainingAlphas/Ensemble/ItemMetadata.ipynb");
@nbinclude("SuppressImplicit.ipynb");

In [ ]:
function compute_nonlinear_alpha()
    params = read_params(source)
    base_features = reduce(
        hcat,
        read_recommendee_suppressed_alpha(x, "all", false).rating for x in params["alphas"]
    )
    metadata_features = get_xgboost_features(fill(1, num_items()), collect(1:num_items()))
    X = hcat(base_features, metadata_features)

    preds = vec(predict(params["model"], X))
    write_recommendee_alpha(preds, source)
end;

In [ ]:
function compute_explicit_alpha()
    source = "Explicit"
    alphas = read_params(source)["alphas"]
    preds = zeros(Float32, num_items())
    for alpha in alphas
        preds += read_recommendee_suppressed_alpha(alpha, "all", false).rating
    end
    write_recommendee_alpha(preds, source)
end;

In [ ]:
function compute_alpha()
    compute_nonlinear_alpha()
    compute_explicit_alpha()
end;

In [ ]:
compute_alpha();